jupyter-lab

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math


In [6]:
Data = pd.read_excel(r'C:\Users\Mehdi\Desktop\AllCompS.xlsx')
Data_Results = pd.read_excel(r'C:\Users\Mehdi\Desktop\1.xlsx')
FAH_CU = Data_Results[(Data_Results.form == "SAR")]
Data_Numpy = FAH_CU.to_numpy()
FAH_CU2 = pd.DataFrame(Data_Numpy)

GOR Estimator

In [7]:
xi = []
yi = []
zi = []
Ki = []
GOR = []
diff = []
#Data without n2, co2, h2s
Data2 = Data.iloc[3:, :]
Liq_Dens = 62.4 * (141.5 / (FAH_CU2.loc[:, 7] + 131.5)) 
MW = FAH_CU2.loc[:, 24]
GOR_Real = FAH_CU2.loc[:, 8]
GOR_Real = GOR_Real.to_numpy()
counter = 0
for i in Data.columns:
    Data3 = Data2[i].dropna()
    if i%4 == 1:
        xi.append(Data3)
    elif i%4 == 2:
        yi.append(Data3)
    elif i%4 == 3:
        zi.append(Data3)
        counter += 1      
for i in range(0,counter):
    b, a = np.polyfit(xi[i]/zi[i], yi[i]/zi[i], deg=1)  
    print(b, a)
    GOR.append(379.5 * Liq_Dens[i] * 5.615 / (-b * MW[i]))  #lb/ft3
    diff.append(abs((GOR[i] - GOR_Real[i])*100/GOR_Real[i]))

Dic = {'GOR_cal' : GOR, 'GOR Real' : GOR_Real, 'diff' : diff }
Rep = pd.DataFrame(Dic)

-1.7731040571352217 2.772814096008323
-1.058037036103347 1.7877514946464748
-1.2347968287362832 2.2356568079452255
-1.4371092134940615 2.386771456778511
-1.077631905319742 2.07802478047268
-1.5152277055009584 2.515594082176914
-1.238393388544252 2.238805302603495
-1.2237094279308651 2.224502780626243
-1.1860731458682507 2.1869934535287316
-0.9989524065666101 1.9986349610913496
-0.7375350732862707 1.32017760737808
-1.0335592531129119 2.0339092563147503
-0.8545282179446272 1.9149954357847518
-1.3299505015706736 2.3299371469834464
-1.372470950168425 2.3733430980708765
-1.1575135759546469 2.2105462880639624
-1.438085123638577 2.441316428634011
-1.2380896507210353 2.23841774670488
-1.1106732462546216 2.1110836921783784
-1.1538534222740566 2.1543708003601614
-1.2249710002283132 2.2284548735291563
-1.1795222036343225 2.183561761119069
-1.4334589364910197 2.4318027199880032
-1.1677394809448867 2.1690559546877255
-1.0816918999523102 2.0820780720089984
-1.2238530479565304 2.224574391019679
-1.13

Hoffman plot

In [8]:
Tb = [201.27, 332.51, 416.25, 470.78, 491.08, 542.12, 556.92]
Tc = [343.33, 549.92, 666.06, 734.46, 765.46, 829.1, 845.8]
Pc = [666.4, 706.5, 616, 527.9, 550.6, 490.4, 488.6]
T = 294.3 +460
Psc = 14.7
Psep = 14.7
F = []
Dist = []

#Hoffman factor
for i in range(0, len(Tb)):
    F.append(((1/Tb[i])-(1/T)) * (np.log10(Pc[i]/Psc)) / ((1/Tb[i])-(1/Tc[i])))


for i in range(0,counter):
    Ki = yi[i][0:7]/xi[i][0:7]
    #Plot log(Ki*Psep) Vs Fi
    #plt.figure()
    prod = np.dot(Ki, Psep)
    logprod = np.log10(prod)
    #plt.scatter(F, logprod)
    m, c = np.polyfit(F, np.log10(prod), deg=1)
    xseq = np.linspace(0, 3, num=100)
    Dist.append(np.sum(abs(np.dot(m, F) - logprod + c) / math.sqrt(m**2 + 1)))
    #plt.plot(xseq, c + m * xseq)
    #plt.show()
    

Rep['Hoffman'] = Dist    

Campbell-Buckley

In [9]:
Dist2 = []
xval = np.power(Tc, 2)/100000
for i in range(0,counter):
    Ki = yi[i][0:7]/xi[i][0:7]
    #plt.figure()
    logK = logprod = np.log10(Ki)
    #plt.scatter(xval, logK)
    m, c = np.polyfit(xval, logK, deg=1)
    xseq = np.linspace(0, 8, num=100)
    Dist2.append(np.sum(abs(np.dot(m, xval) - logK + c) / math.sqrt(m**2 + 1)))
    #plt.plot(xseq, c + m * xseq)
    
Rep['Buckley'] = Dist2   
print(Rep)

        GOR_cal GOR Real        diff   Hoffman   Buckley
0    290.408095    403.0   27.938438  0.561023  1.230678
1    394.650075    300.0   31.550025  0.803892  0.971609
2    338.171035   346.48    2.398108  2.375779  4.128409
3    326.318013    308.0    5.947407  2.293264  4.002147
4    359.128202   432.75   17.012547  0.295678  0.784658
5    246.624852   427.84   42.355822  0.649039  0.791649
6    306.371057   372.52   17.757152  0.586208  1.067598
7    302.079671   370.58   18.484627  0.548433  1.063205
8    371.036279    314.2   18.089204  0.853653  2.025610
9    445.397407    405.6    9.811984  0.514021  1.009954
10   558.119670    383.3   45.609097  1.003224  1.651336
11   423.419527    395.3    7.113465  0.474342  1.005441
12   512.098450   462.94   10.618752  0.474146  1.011475
13   306.707373    337.0    8.988910  0.517095  0.973705
14   338.144208    357.0    5.281734  0.535975  0.913183
15   352.915890    335.0    5.348027  0.789949  1.346165
16   343.400635   362.91    5.3